In [ ]:
# Création du dossier WebCrawler contenant l'ensemble des fichiers utiles au fonctionnement de scrapy
!scrapy startproject MangaCrawler

In [ ]:
# ***************** création du projet Manga *********

!cd MangaCrawler && scrapy genspider mangascantrad https://www.japscan.me/

In [ ]:
# ***************** Ajout au fichier items.py & Liste des données qui seront récupérés par manga *********

class MangacrawlerItem(scrapy.Item):
    # define the fields for your item here like:
    title = scrapy.Field()
    img = scrapy.Field()
    rating = scrapy.Field()
    last_chapter = scrapy.Field()
    link = scrapy.Field()
    genres = scrapy.Field()
    published_date = scrapy.Field()
    state = scrapy.Field()
    nb_comments = scrapy.Field()
    pass

In [ ]:
# ***************** Liste des commandes********************

#1. Le titre du manga
response.css('div#main div.card-body h1::text').get()

#2. Le lien de l'image du manga
response.css('div#main div.card-body img').attrib['src']

#3. L'origine du manga
response.css('div#main div.card-body p.mb-2')[1].css('span::text')[1].get()

#4. La description du manga
response.css('div#main div.card-body p.list-group-item.list-group-item-primary.text-justify::text').get()

#5. Le dernier chapitre du manga
response.css('div#chapters_list div.collapse.show div.chapters_list.text-truncate').get().split(':')[0].split('\t')[-1]

#5. Le lien de la page détail du manga
response.request.url

#8. Les genres du manga
response.css('div#main div.card-body p.mb-2')[5].get().split('</span')[1].split('\t')[7].strip()

#6. Le type du manga
response.css('div#main div.card-body p.mb-2')[4].get().split('</span')[1].split('\t')[7].strip()

#6. La date de publication du manga
response.css('div#main div.card-body p.mb-2')[3].get().split('</span')[1].split('\t')[7].strip()

#7. L'état du manga
response.css('div#main div.card-body p.mb-2')[2].get().split('</span')[1].split('\t')[6].strip()


In [ ]:
# La class MangascantradSpider
import scrapy
from scrapy import Request
from MangaCrawler.items import MangaGenres, MangacrawlerItem, DataBase
import sqlalchemy as db
from sqlalchemy.orm import declarative_base, relationship

class MangascantradSpider(scrapy.Spider):
    name = 'mangascantrad'
    allowed_domains = ['manga-scantrad.net']

    # Liste des urls par pages
    start_urls = [f'http://manga-scantrad.net/manga/page/{n}' for n in range(1, 31)]

    # Création de la base de données
    database = DataBase('database_manga')

    # Creation des tables avec une relation ManyToMany
    Base = declarative_base()
    association_table = db.Table(
        "mangas_assoc_genres",
        Base.metadata,
        db.Column("mangas_id", db.ForeignKey("mangas.id_")),
        db.Column("mangas_genres_id", db.ForeignKey("mangas_genres.id_")),
    )
    database.create_table('mangas_genres',
                          id_=db.Integer,
                          name=db.String,
                          )

    database.create_table('mangas',
                          id_=db.Integer,
                          title=db.String,
                          img=db.String,
                          rating=db.String,
                          last_chapter=db.String,
                          link=db.String,
                          genres=db.String,
                          published_date=db.String,
                          state=db.String,
                          nb_comments=db.String,
                          children=relationship("mangas_genres", secondary=association_table)
                          )

    def start_requests(self):
        print("iok")
        # for url in self.start_urls:
        #     yield Request(url=url, callback=self.parse_manga)

    def parse_manga(self, response):
        mangas = response.css('div.js-categories-seasonal.js-block-list.list table tr')[1:]
        for manga in mangas:
            item = MangacrawlerItem()

            # Nom manga
            try:
                item['name'] = manga.css('td')[1].css('a.hoverinfo_trigger.fw-b strong::text').get()
            except:
                item['name'] = 'None'

            # Image manga
            try:
                item['img'] = manga.css('td')[0].css('a.hoverinfo_trigger img').attrib['data-src']
            except:
                item['img'] = 'None'

            # Description manga
            try:
                item['description'] = manga.css('td')[1].css('div.pt4::text').get()
            except:
                item['description'] = 'None'

            # Ajouter dans la base de données
            self.database.add_row('manga',
                                  name=item['name'],
                                  img=item['img'],
                                  description=item['description']
                                  )
            yield item



In [ ]:
# Exécution de scrapy
!cd MangaCrawler/MangaCrawler/spiders && scrapy crawl mangascantrad -o mangascantrad.csv

*********Table : 'mangas_types' are created succesfully************
*********Table : 'mangas' are created succesfully************
*********Table Relationship : 'mangas_assoc_types' are created succesfully************
2022-10-20 19:03:43 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: MangaCrawler)
2022-10-20 19:03:43 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.9.2 (default, Feb 28 2021, 17:03:44) - [GCC 10.2.1 20210110], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.0-19-amd64-x86_64-with-glibc2.31
2022-10-20 19:03:43 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'MangaCrawler',
 'NEWSPIDER_MODULE': 'MangaCrawler.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['MangaCrawler.spiders']}
2022-10-20 19:03:43 [py.warnings] WARNING: /home/yannis/.local/lib/python3.9/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is

2022-10-20 19:03:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/10-count/> (referer: None)
2022-10-20 19:03:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/10-years-in-friend-zone/> (referer: None)
2022-10-20 19:03:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/1-24-romance/> (referer: None)
Row id added
2022-10-20 19:03:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/07-ghost/>
{'description': 'Il y a de cela plus de mille ans, le monde était divisé en '
                'deux puissants pays de force égale : l’empire de Barsburg, '
                'sous la protection divine de l’Œil du dieu Raphaël, et le '
                'royaume de Raggs, protégé par l’Œil du dieu Michael. Lorsque '
                'Raggs fut anéanti par la puissance prodigieuse de Barsburg, '
                'le destin du monde bascula, en même temps que celui de Teito '
                'Kle

Row id added
2022-10-20 19:03:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/100-gokuama-kareshi/>
{'description': "Petite anthologie d'histoires écrites par différentes "
                'auteures.\r\n'
                '1. Kodomo Jya, Naino - Ririko Yoshioka\r\n'
                "Nano et Ryou sont deux amis d'enfances très proche l'un "
                "l'autre. Quand ils était jeunes Nano promis à Ryou de le "
                'protéger de tous les dangers comme le ferait une grande sœur. '
                'Mais quelques années plus tard, les rôles se sont peut-être '
                'inversés.\r\n'
                '2. Natsu ga Chikai - Ayako Kimura\r\n'
                'Asamizu aime Tsuji Hideto et, afin de pouvoir lui parler tous '
                "les jours, elle viole délibérément les règlements de l'école "
                'afin de lui convenir.\r\n'
                '3. HOLE - Kirameku Sekai - Rie Takahashi\r\n'
                "Il n'arrêtait pas d

Row id added
2022-10-20 19:03:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/10-count/>
{'description': 'Shirotani est un jeune homme brillant atteint de mysophobie. '
                'Quant le chef d’entreprise pour lequel il travaille est '
                'victime d’un accident, il rencontre Kurose, un jeune homme '
                'qui lui conseille de se faire soigner et lui laisse la carte '
                'd’une clinique psychiatrique. Le lendemain, Shirotani se rend '
                'à la clinique et comprend que Kurose est un des psychiatres '
                'qui y travaillent. Les deux noueront alors une relation qui '
                'dépassera bientôt le cadre médecin-patient...',
 'genres': 'Drame, Tranche De Vie, Romance, Psychologie, Adulte, Gay-Lesbien, '
           'Smut, Homosexualité',
 'img': '/imgs/mangas/10-count.jpg',
 'last_chapter': 'https://www.japscan.me/manga/10-count/',
 'link': 'https://www.japscan.me/manga/10-count/'

Row id added
2022-10-20 19:03:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/17h05/>
{'description': 'En raison de certaines circonstances, un aspirant artiste de '
                'bande dessinée doit travailler à l’école la nuit, malgré les '
                'histoires d’un fantôme qui hante la salle de classe. Comme '
                'par hasard, ils se rencontrent.',
 'genres': 'Fantastique, Mystère, School Life, Amitié, Webtoon',
 'img': '/imgs/mangas/17h05.jpg',
 'last_chapter': 'https://www.japscan.me/manga/17h05/',
 'link': 'https://www.japscan.me/manga/17h05/',
 'origin': 'Japon',
 'published_date': '2019',
 'state': 'En Cours',
 'title': 'Manga 17H05',
 'type': 'None'}
Row id added
2022-10-20 19:03:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/16-life/>
{'description': 'Saya est une jeune fille de 16 ans qui souhaite rester '
                'discrète, estimant que la «lumière» est réservée à sa grande '
       

...

Row id added
2022-10-20 19:04:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/assassins-pride/>
{'description': "L'histoire se déroule dans un monde où seuls les Nobles sont "
                "capables d'utiliser le Mana, ils sont donc les seuls à "
                'pouvoir défendre villes et villages de ce qui les menacent. '
                'Merida est une jeune fille exceptionnelle, mais dans le '
                'mauvais sens du terme : bien que née noble, elle est '
                "incapable d'utiliser le Mana. Le jeune Kûfa est choisi pour "
                "l'aider à développer son potentiel... ou l'assassiner si elle "
                'demeure incompétente.',
 'genres': 'Fantastique, Drame, Mystère, Action, Comédie, Magie, Romance, '
           'Harem, Arts Martiaux',
 'img': '/imgs/mangas/assassins-pride.jpg',
 'last_chapter': 'https://www.japscan.me/manga/assassins-pride/',
 'link': 'https://www.japscan.me/manga/assassins-pride/',
 'origin': 'J

Row id added
2022-10-20 19:04:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/a-story-about-a-creepy-girl-smile/>
{'description': 'À venir...',
 'genres': 'School Life, Ecole, Amitié, Romance, Doujinshi, Slice Of Life',
 'img': '/imgs/mangas/a-story-about-a-creepy-girl-smile.jpg',
 'last_chapter': 'https://www.japscan.me/manga/a-story-about-a-creepy-girl-smile/',
 'link': 'https://www.japscan.me/manga/a-story-about-a-creepy-girl-smile/',
 'origin': 'Japon',
 'published_date': '2020',
 'state': 'En Cours',
 'title': 'Manga A Story About A Creepy Girl Smile',
 'type': 'None'}
2022-10-20 19:04:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/astral-project/> (referer: None)
2022-10-20 19:04:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/astra-lost-in-space/> (referer: None)
Row id added
2022-10-20 19:04:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/a-story-about-tr

Row id added
2022-10-20 19:04:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/astro-boy/>
{'description': "En l'an 2003, le monde est couvert par la technologie et la "
                'science, et les robots vivent en société avec les hommes. Au '
                'Japon, le respectable Docteur Tenma est une figure dominante '
                'des sciences, et a contribué à la modernisation du pays. Un '
                'jour, son fils unique, Tobio, se fait tuer dans un accident '
                'de voiture. Pris de désespoir et ne pouvant accepter cette '
                "perte, Tenma décida de faire bâtir un robot à l'image de son "
                'fils afin de le remplacer, et fut muni de super-pouvoirs pour '
                "qu'il ne soit plus victime de quelque accident. Bien que le "
                "nouveau Tobio s'humanisa et développa l'âme d'un garçon "
                "humain, Tenma se rendit compte qu'un robot ne pourrait jamais "
        

Row id added
2022-10-20 19:04:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/a-taste-of-love/>
{'description': "S'il y a bien une chose dont Xia Meng raffole, c'est de "
                'regarder des séries télé encore et encore, surtout la nuit, '
                "et c'est encore mieux si c'est accompagné de bonnes "
                'pâtisseries ! Et oui, notre héroïne est une sacrée gloutonne '
                '! Mais sa gourmandise va la pousser à sortir en pleine nuit, '
                'une nuit tragique où elle va trouver la mort... Et si la mort '
                "n'était pas la fin de tout ? Quel est ce monde étrange où "
                'elle arrive ? Uh ? De la magie ? Des paysages '
                "extraordinaires... Où est-ce qu'elle est là...?",
 'genres': 'Fantastique, Magie, Romance, Amour, Isekai, Webtoon, Fantasy, '
           'Cooking',
 'img': '/imgs/mangas/a-taste-of-love.jpg',
 'last_chapter': 'https://www.japscan.me/manga/a-taste-o

Row id added
2022-10-20 19:04:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/beach-stars/>
{'description': "Pas facile d'être fan de volley-ball quand on joue dans la "
                "catégorie rase-mottes et qu'on n'est que trois à pratiquer "
                "dans son lycée ! Tel est l'amer constat que fait Iruka Nanase "
                'quand le club de son école ferme faute de combattants. Mais '
                "voilà qu'elle découvre par hasard une variante à sa portée et "
                'une championne à qui se mesurer...',
 'genres': 'School Life, Ecchi, Tranche De Vie, Comédie, Tournois, '
           'Volley-ball, Sports',
 'img': '/imgs/mangas/beach-stars.jpg',
 'last_chapter': 'https://www.japscan.me/manga/beach-stars/',
 'link': 'https://www.japscan.me/manga/beach-stars/',
 'origin': 'Japon',
 'published_date': '2007',
 'state': 'Terminé',
 'title': 'Manga Beach Stars',
 'type': 'Shōnen, Seinen'}


In [12]:
# ***************** Afficher les données csv avec pandas *******************

import pandas as pd

df = pd.read_csv('MangaCrawler/MangaCrawler/spiders/mangascantrad.csv')
df

,description,genres,img,last_chapter,link,origin,published_date,state,title,type
0,Le duo de mangaka croisé dans 20th Century Bo...,"Suspense, Drame, Mystère, Surnaturel, Post-Apo...",/imgs/mangas/20-seiki-shounen-no-wakiyaku-ujik...,https://www.japscan.me/manga/20-seiki-shounen-...,https://www.japscan.me/manga/20-seiki-shounen-...,Japon,2010,Terminé,Manga 20 Seiki Shounen No Wakiyaku - Ujiko Uji...,Seinen
1,Le jeune Nathan devient un garçon retiré et as...,"Tranche De Vie, Comédie, Webtoon",/imgs/mangas/20-twenty.jpg,https://www.japscan.me/manga/20-twenty/,https://www.japscan.me/manga/20-twenty/,Japon,2018,En Cours,Manga 20 Twenty,Shounen Ai
2,Kenji est un jeune homme tenant la boutique de...,"Suspense, Drame, Mature, Mystère, Action, Amit...",/imgs/mangas/20th-century-boys.jpg,https://www.japscan.me/manga/20th-century-boys/,https://www.japscan.me/manga/20th-century-boys/,Japon,1999,Terminé,Manga 20th Century Boys,Seinen
3,Higurashi Tsugumi a perdu ses parents et sa go...,"Drame, Surnaturel, Comédie, Romance, Amour, Fa...",/imgs/mangas/25-ji-no-ghost-writer.jpg,https://www.japscan.me/manga/25-ji-no-ghost-wr...,https://www.japscan.me/manga/25-ji-no-ghost-wr...,Japon,2019,En Cours,Manga 25-Ji No Ghost Writer,Seinen
4,La suite et fin de la série 20th Century Boys....,"Suspense, Drame, Mature, Mystère, Action, Trag...",/imgs/mangas/21st-century-boys.jpg,https://www.japscan.me/manga/21st-century-boys/,https://www.japscan.me/manga/21st-century-boys/,Japon,2007,Terminé,Manga 21st Century Boys,Seinen
...,...,...,...,...,...,...,...,...,...,...
813,"Au XXIè siècle, alors que leur existence avait...","Action, Aventure, Monstres, Animaux, Monster G...",/imgs/mangas/bna-brand-new-animal.jpg,https://www.japscan.me/manga/bna-brand-new-ani...,https://www.japscan.me/manga/bna-brand-new-ani...,Japon,None,En Cours,Manga BNA: Brand New Animal,Seinen
814,La paix du monde est menacée : l'Empire Chauve...,"Surnaturel, Action, Comédie, Aventure, Science...",/imgs/mangas/bobobo-bo-bo-bobo.jpg,https://www.japscan.me/manga/bobobo-bo-bo-bobo/,https://www.japscan.me/manga/bobobo-bo-bo-bobo/,Japon,2001,En Cours,Manga Bobobo-Bo Bo-Bobo,Shōnen
815,Kai a une politique de ne jamais utiliser son ...,"Action, Comédie, Aventure, Science-fiction",/imgs/mangas/blust.jpg,https://www.japscan.me/manga/blust/,https://www.japscan.me/manga/blust/,Japon,2009,Terminé,Manga Blust!,Shōnen
816,Un hikikomori reçoit un SMS anonyme qui dit 'A...,"Suspense, Fantastique, Action, Horreur, Tragéd...",/imgs/mangas/bocchiman.jpg,https://www.japscan.me/manga/bocchiman/,https://www.japscan.me/manga/bocchiman/,Japon,2014,En Pause,Manga Bocchiman,Shōnen
